# Veeper: Create, Run, and Collate

This is a generalized code that can bring in any spectra, save it to its own components folder, and save the corresponding fits

Install: https://github.com/mattcwilde/veeper


Here are some notes for the future

#### Running this code in Step 3 will produce a FitInspection.pdf file with a preview of your voigt fits: 
- Red line = total 
- dashed line = particular component 
- the order is the redshift 

#### If the file needs to be changed (editing.ipynb): 

In [1]:
##--- Step one 
from joebvp.utils import pyigm_to_veeper
from joebvp.VPmeasure import batch_fit
from joebvp import VPmeasure
import glob
import os
import astropy.units as u

read_sets: Using set file -- 
  /Users/samgarza/linetools/linetools/lists/sets/llist_v1.3.ascii
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/morton03_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/verner96_tab1.fits.gz
joebvp.makevoigt: No local joebvp_cfg.py found, using default cfg.py file from joebvp.
joebvp.VPmeasure: No local joebvp_cfg.py found, using default cfg.py file from joebvp.
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/morton03_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/verner96_tab1.fits.gz
joebvp.makevoigt: No local joebvp_cfg.py found, usin

#### Paths for each of my spectra: 
- ##### NGC 3489 / J1101: 
    - json_path =  '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1101+1429/SamIDs_Nov28_2022.json' 
    - norm = 'J1101+1429_nbin3_jwnorm.fits'

- ##### NGC 4026 / J1159: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159+5106/SamIDs_meth3_Nov28_2022.json'
    - norm = 'J1159+5106_nbin3_jwnorm_meth3.fits'
    
- ##### NGC 4736 / J1249: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249+4122/SamIDs_Nov28_2022.json'
    - norm = 'J1249+4122_nbin3_visit12_jwnorm.fits'
    
- ##### NGC 3414 / 1051: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1051+2805/SamIDs_Nov29_2022.json'
    - norm = 'J1051+2805_nbin3_jwnorm.fits'
    
- ##### NGC 3627 / J1123: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123+1257/SamIDs_Nov29_2022.json'
    - norm = 'J1123+1257_nbin3_BASIC_jwnorm.fits'

- ##### NGC 1097 / J0246: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246-3007/SamIDs_Nov28_2022.json'
    - norm = 'J0246-3007_nbin3_allvisits_jwnorm.fits'

- ##### NGC 4564 / J1235: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235+1123/SamIDs_Nov28_2022.json'
    - norm = 'J1235+1123_twovisits_m3_jwnorm.fits'

In [ ]:
#Step 2: Imput the specifications 
    # basically takes the gueses and the norm to start the process
    #dv_expand is how much you can expand the redlines to be of some problem 

#These will change for each galaxy/line of sight  
#path to specific .json file
json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123+1257/SamIDs_Nov29_2022.json'

# normalization file that you want to use
norm = 'J1123+1257_nbin3_BASIC_jwnorm.fits'
norm_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123+1257/J1123+1257_nbin3_BASIC_jwnorm.fits' #path to normalization file 

pyigm_to_veeper(json_path, norm, dv_expand=50*u.km/u.s)

In [ ]:
input_files = glob.glob('component_groups/*.txt')

#This was 10 in the other one but I want to make this 20 since it skipped something that was 15 before 
max_len = 20 #skip over files where there are more than this many lines to fit 
redo = False

to_do = []
for input_file in input_files:
    if 'rchi2' in input_file:
        continue
    if not redo:
        #already done?
        if os.path.exists(input_file.replace('.txt', '.VP')): 
            print('Skipping: Already done {}'.format(input_file))
            continue
    
    #too long?
    with open(input_file, 'r') as f:
        length = len(f.readlines())
        if length > max_len:
            print('Skipping: Too long ({} rows) {}'.format(length, input_file))
            continue
            
    to_do.append(input_file)

In [ ]:
###--- Step 3: Run the fit
VPmeasure.batch_fit(norm_path, filelist=to_do, filepath='./component_groups/')

In [ ]:
#make the directory to hold creates files for each qso/LOS 

parent_path = '/Users/samgarza/Dropbox/Mac/Documents/COS-Holes/veeper_and_voigt/' #where I want the created files to be put in general 
qso_name = 'J1123/' #need to change this to match each QSO LOS name; where I want the created files to be put in specifically 

#set directory name
directory = qso_name
# Path
path = os.path.join(parent_path, directory)
#Make it happen
os.mkdir(path)

In [ ]:
#Step 4: Collate
import pandas as pd

vp_file_list = glob.glob('component_groups/*.VP')
vp_file_list

In [ ]:
vp_dfs = []

for vp_file in vp_file_list:
    vp_df = pd.read_csv(vp_file, sep='|')
    vp_dfs.append(vp_df)
    
all_vp_df = pd.concat(vp_dfs, ignore_index=True)
all_vp_df

In [ ]:
#Save collate informtion to csv file 
all_vp_df.to_csv(path + 'all_vp.csv')

In [ ]:
import shutil

#Move over the component_groups
shutil.move(parent_path + 'component_groups/', path)

In [ ]:
#list of created files that need to be moved over to QSO LOS specific directory 
files_to_move = ['_VP_log.dat', 'all_VP.txt', 'compiledVPoutputs.dat', 'FitInspection.fits', 'FitInspection.pdf']

# iterate files
for file in files_to_move:
    # construct full file path
    source = parent_path + file
    destination = path + file
    # move file
    shutil.move(source, destination)
    print('Moved:', file)